In [152]:
import numpy as np
import pandas as pd
#hyper-parameters
INPUT_LAYER_SIZE=784
OUTPUT_LAYER_SIZE=10


In [153]:
train=pd.read_csv('./data/train.csv')
val=pd.read_csv('./data/val.csv')
test=pd.read_csv('./data/test.csv')

In [322]:
training_data_class =train.iloc[:,-1].copy().as_matrix()
training_data=train.iloc[:,1:785].copy().as_matrix()

In [155]:
validating_data_class =val.iloc[:,-1].copy().as_matrix()
validating_data=val.iloc[:,1:785].copy().as_matrix()

model=nn.MLPClassifier(activation='logistic',hidden_layer_sizes=(784,400,200,100,50,25),)
model.fit(training_data[:100],training_data_class[:100])
result=model.predict(training_data[:100])
r=training_data_class[:100]-result
print 1.0*np.count_nonzero(r)/len(validating_data_class[:100])

In [291]:
def create_output(class_labels):
    output=np.zeros((len(class_labels),10))
    for i in range(len(class_labels)):
        output[i,class_labels[i]]=1
    return output
def sigmoid(value):
    return 1/(1+np.exp(-value))
def tanh(value):
    return np.tanh(value)
def sigmoid_dif(value):
    return sigmoid(value)*(1-sigmoid(value))
def tanh_dif(value):
    return 1-tanh(value)*tanh(value)
def softmax(vector):
    num= np.exp(vector)
    return num/np.sum(num)
def paraCopy(w,b,multiplier=1):
    return [i*multiplier for i in w],[i*multiplier for i in b]
def paraAdd(A,B):
    a,b=A
    c,d=B
    return [i+j for i,j in zip(a,c)],[i+j for i,j in zip(b,d)]

In [292]:
def shaper(lst):
    for i in lst:
        print 'shape -',i.shape
def loss(list1,list2):
    return -np.sum([j*np.log2(i) for i,j in zip(list1,list2)])

In [394]:
class NN_Model:
    def __init__(self,lr=0.01,momentum=0,hidden_layers=(100,),activation='sigmoid',loss='sq',
                 opt='gd',batch_size=1,anneal=False,save_dir='./temp/',expt_dir='./temp/',max_iter=50):
        self.lr=lr
        self.momentum=momentum
        self.layers=(INPUT_LAYER_SIZE,)+hidden_layers+(OUTPUT_LAYER_SIZE,)
        if activation=='sigmoid':
            self.activation=sigmoid
        elif activation=='tanh':
            self.activation=tanh
        else:
            print 'Error : activation function not found'
                
        
        self.loss=loss
        self.opt=opt
        self.batch_size=batch_size
        self.anneal=anneal
        self.save_dir=save_dir
        self.expt_dir=expt_dir
        self.num_layers=len(self.layers)
        self.max_iter=max_iter
    
    def __forward_propagation(self):
        h_set=[]
        a_set=[]
        h=self.input_data[TRAINER]
        h_set.append(h) #experimental
        L=self.num_layers-2 # 2 are input and output layers
        for k in range(L): 
            a=self.biases[k]+np.matmul(self.weights[k],h)
            h=self.activation(a)
            a_set.append(a)
            h_set.append(h)
        a=self.biases[L]+np.matmul(self.weights[L],h)
        a_set.append(a)
        y=softmax(a)
        #h_set.append(y) #experimental
        #print h.shape
            
        return h_set,a_set,y
    
    def __forward_propagation_test(self,h):
        L=self.num_layers-2 # 2 are input and output layers
        for k in range(L): 
            a=self.biases[k]+np.matmul(self.weights[k],h)
            h=self.activation(a)
        a=self.biases[L]+np.matmul(self.weights[L],h)
        y=softmax(a)
        #h_set.append(y) #experimental
        #print h.shape
            
        return y
    
    
    def __back_propagation(self,h,a,y):
        L=self.num_layers-2
        dWeights=[]
        dBiases=[]
        
        daL_loss=-(self.output_data[TRAINER]-y) # for cross-entropy loss function
        dA_loss=daL_loss
        for k in range(L,-1,-1):
            #print 'k=',k
            dW_loss=np.outer(dA_loss,h[k]).T
            dB_loss=dA_loss
            
            if k!=0:
                dH_loss=np.matmul(self.weights[k].T,dA_loss)
            
                dA_loss=dH_loss*sigmoid_dif(a[k-1])
            
            
            dWeights=[dW_loss.T]+dWeights
            dBiases=[dB_loss.T]+dBiases
            #print dA_loss.shape
            #return _,_
            
            
        return dWeights,dBiases
    
    def __update(self,dWeights,dBiases):
        for i in range(len(self.weights)):
            self.weights[i]=self.weights[i]-dWeights[i]
            self.biases[i]=self.biases[i]-dBiases[i]

    
    
    
    def __train(self):
        global TRAINER
        for i in range(self.max_iter):
            
            accuracy=0
            if self.batch_size==1:
                self.batch_size=len(training_data)
            
            sets=len(training_data)/self.batch_size # number of batches
                
            oldDWeights,oldDBiases=paraCopy(self.weights,self.biases,0) # for momentum
            for j in range(sets):
                #print 'SET=',j
                los=0
                dWeights,dBiases=paraCopy(self.weights,self.biases,0)
                for TRAINER in range(j*self.batch_size,(j+1)*self.batch_size):
                    #print 'TRAINER=',TRAINER
                    h,a,y=self.__forward_propagation()

                    
                    dWeights,dBiases=paraAdd(self.__back_propagation(h,a,y),
                                         paraCopy(dWeights,dBiases,1))
                    los+=loss(y,self.output_data[TRAINER])

                dWeights,dBiases=paraAdd(paraCopy(oldDWeights,oldDBiases,self.momentum),
                                         paraCopy(dWeights,dBiases,self.lr))

                self.__update(dWeights,dBiases)


                
                oldDWeights,oldDBiases=paraCopy(dWeights,dBiases,1)

                #print np.argmax(y),self.raw_class_labels[TRAINER]
                #if np.argmax(y)==self.raw_class_labels[TRAINER]:

                 #   accuracy+=1
                print 'Epoch : ',i,'Step : ',j,'loss : ',los
                #print 'acc=',accuracy*1.0/sets

    def fit(self,X,Y):
        self.input_data=X
        self.output_data=create_output(Y)
        self.weights=[]
        self.biases=[]
        self.raw_class_labels=Y
        for i in range(self.num_layers-1):
            self.weights.append(np.random.randint(-10,10,size=(self.layers[i+1],self.layers[i])))
            self.biases.append(np.random.randint(-10,10,size=(self.layers[i+1])))
            
        self.__train()
        
    def predict(self,X):
        return [np.argmax(self.__forward_propagation_test(i)) for i in X]    
        
    

In [403]:
model=NN_Model(hidden_layers=(128,128),lr=0.001,max_iter=10,momentum=0.5,batch_size=25)
training_data=np.array(training_data)

In [404]:
model.fit(training_data,training_data_class)


/home/mak/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys


Epoch :  0 Step :  0 loss :  3318.59321019
Epoch :  0 Step :  1 loss :  2559.03537236
Epoch :  0 Step :  2 loss :  2962.23578898
Epoch :  0 Step :  3 loss :  3223.16545439
Epoch :  0 Step :  4 loss :  2494.63593439
Epoch :  0 Step :  5 loss :  2431.80315548
Epoch :  0 Step :  6 loss :  2669.44497773
Epoch :  0 Step :  7 loss :  2741.99675251
Epoch :  0 Step :  8 loss :  2230.07437052
Epoch :  0 Step :  9 loss :  2279.94321488
Epoch :  0 Step :  10 loss :  2285.17734452
Epoch :  0 Step :  11 loss :  2662.86170082
Epoch :  0 Step :  12 loss :  1994.66193094
Epoch :  0 Step :  13 loss :  2470.982484
Epoch :  0 Step :  14 loss :  2465.38258064
Epoch :  0 Step :  15 loss :  2197.15886095
Epoch :  0 Step :  16 loss :  1675.2801181
Epoch :  0 Step :  17 loss :  3261.71066763
Epoch :  0 Step :  18 loss :  2367.78338379
Epoch :  0 Step :  19 loss :  1975.83151618
Epoch :  0 Step :  20 loss :  2320.38228427
Epoch :  0 Step :  21 loss :  2814.08161568
Epoch :  0 Step :  22 loss :  2511.31681973
E

Epoch :  0 Step :  185 loss :  1092.47802263
Epoch :  0 Step :  186 loss :  1185.97056804
Epoch :  0 Step :  187 loss :  911.98610403
Epoch :  0 Step :  188 loss :  1161.1483816
Epoch :  0 Step :  189 loss :  1237.42812658
Epoch :  0 Step :  190 loss :  898.332264238
Epoch :  0 Step :  191 loss :  1106.21358485
Epoch :  0 Step :  192 loss :  955.273873214
Epoch :  0 Step :  193 loss :  1396.16922872
Epoch :  0 Step :  194 loss :  871.49329975
Epoch :  0 Step :  195 loss :  1172.23508932
Epoch :  0 Step :  196 loss :  936.583010652
Epoch :  0 Step :  197 loss :  1322.8327189
Epoch :  0 Step :  198 loss :  1363.57728329
Epoch :  0 Step :  199 loss :  1134.12715332
Epoch :  0 Step :  200 loss :  1001.38269776
Epoch :  0 Step :  201 loss :  1203.81735094
Epoch :  0 Step :  202 loss :  1213.58589318
Epoch :  0 Step :  203 loss :  704.954696611
Epoch :  0 Step :  204 loss :  973.495438715
Epoch :  0 Step :  205 loss :  1093.61840782
Epoch :  0 Step :  206 loss :  981.867607705
Epoch :  0 Ste

Epoch :  0 Step :  369 loss :  948.280368627
Epoch :  0 Step :  370 loss :  754.672694478
Epoch :  0 Step :  371 loss :  1019.00456285
Epoch :  0 Step :  372 loss :  628.985904268
Epoch :  0 Step :  373 loss :  805.890142719
Epoch :  0 Step :  374 loss :  772.522170464
Epoch :  0 Step :  375 loss :  769.329124073
Epoch :  0 Step :  376 loss :  678.573030229
Epoch :  0 Step :  377 loss :  866.976521239
Epoch :  0 Step :  378 loss :  675.715051681
Epoch :  0 Step :  379 loss :  651.292189602
Epoch :  0 Step :  380 loss :  687.940410144
Epoch :  0 Step :  381 loss :  769.134171536
Epoch :  0 Step :  382 loss :  466.532041464
Epoch :  0 Step :  383 loss :  875.538327186
Epoch :  0 Step :  384 loss :  703.943792714
Epoch :  0 Step :  385 loss :  455.598251678
Epoch :  0 Step :  386 loss :  574.351243272
Epoch :  0 Step :  387 loss :  756.514258295
Epoch :  0 Step :  388 loss :  576.93792263
Epoch :  0 Step :  389 loss :  544.431753358
Epoch :  0 Step :  390 loss :  759.564501793
Epoch :  0 

Epoch :  0 Step :  552 loss :  740.977797885
Epoch :  0 Step :  553 loss :  824.376646823
Epoch :  0 Step :  554 loss :  755.287085292
Epoch :  0 Step :  555 loss :  715.081931927
Epoch :  0 Step :  556 loss :  618.326958381
Epoch :  0 Step :  557 loss :  564.048018741
Epoch :  0 Step :  558 loss :  464.967218072
Epoch :  0 Step :  559 loss :  527.214055479
Epoch :  0 Step :  560 loss :  540.373516928
Epoch :  0 Step :  561 loss :  684.761722852
Epoch :  0 Step :  562 loss :  630.347445725
Epoch :  0 Step :  563 loss :  540.830259892
Epoch :  0 Step :  564 loss :  621.348113542
Epoch :  0 Step :  565 loss :  520.366629683
Epoch :  0 Step :  566 loss :  590.720293121
Epoch :  0 Step :  567 loss :  537.589268457
Epoch :  0 Step :  568 loss :  552.979379474
Epoch :  0 Step :  569 loss :  410.49841759
Epoch :  0 Step :  570 loss :  898.057163003
Epoch :  0 Step :  571 loss :  661.362545752
Epoch :  0 Step :  572 loss :  663.962735214
Epoch :  0 Step :  573 loss :  504.36419385
Epoch :  0 S

...

Epoch :  9 Step :  1365 loss :  57.7667613265
Epoch :  9 Step :  1366 loss :  62.9383105327
Epoch :  9 Step :  1367 loss :  50.1601404405
Epoch :  9 Step :  1368 loss :  60.2718947902
Epoch :  9 Step :  1369 loss :  43.9566811441
Epoch :  9 Step :  1370 loss :  55.4273005224
Epoch :  9 Step :  1371 loss :  53.4771206493
Epoch :  9 Step :  1372 loss :  44.1432565602
Epoch :  9 Step :  1373 loss :  45.1468896502
Epoch :  9 Step :  1374 loss :  45.1750545158
Epoch :  9 Step :  1375 loss :  56.6250374115
Epoch :  9 Step :  1376 loss :  69.598257666
Epoch :  9 Step :  1377 loss :  75.9315215942
Epoch :  9 Step :  1378 loss :  46.6507614264
Epoch :  9 Step :  1379 loss :  48.7019186984
Epoch :  9 Step :  1380 loss :  40.1057860531
Epoch :  9 Step :  1381 loss :  42.680018021
Epoch :  9 Step :  1382 loss :  57.8517142372
Epoch :  9 Step :  1383 loss :  53.0487317939
Epoch :  9 Step :  1384 loss :  120.596764101
Epoch :  9 Step :  1385 loss :  54.8254299284
Epoch :  9 Step :  1386 loss :  52.3

Epoch :  9 Step :  1546 loss :  41.4980150554
Epoch :  9 Step :  1547 loss :  125.157106055
Epoch :  9 Step :  1548 loss :  40.6374481472
Epoch :  9 Step :  1549 loss :  58.5525454196
Epoch :  9 Step :  1550 loss :  35.8851946878
Epoch :  9 Step :  1551 loss :  44.5456388747
Epoch :  9 Step :  1552 loss :  76.2739375589
Epoch :  9 Step :  1553 loss :  49.0653282985
Epoch :  9 Step :  1554 loss :  39.7455954386
Epoch :  9 Step :  1555 loss :  38.351469058
Epoch :  9 Step :  1556 loss :  36.3247204992
Epoch :  9 Step :  1557 loss :  51.4270673091
Epoch :  9 Step :  1558 loss :  42.3108411137
Epoch :  9 Step :  1559 loss :  101.991914126
Epoch :  9 Step :  1560 loss :  43.1536746289
Epoch :  9 Step :  1561 loss :  46.5201983202
Epoch :  9 Step :  1562 loss :  78.3473802891
Epoch :  9 Step :  1563 loss :  40.3542437301
Epoch :  9 Step :  1564 loss :  30.5022316578
Epoch :  9 Step :  1565 loss :  90.3911472792
Epoch :  9 Step :  1566 loss :  108.09986514
Epoch :  9 Step :  1567 loss :  38.1

Epoch :  9 Step :  1725 loss :  45.7703085803
Epoch :  9 Step :  1726 loss :  41.7596918675
Epoch :  9 Step :  1727 loss :  45.8429498226
Epoch :  9 Step :  1728 loss :  46.1895629915
Epoch :  9 Step :  1729 loss :  65.0918687999
Epoch :  9 Step :  1730 loss :  49.5726006069
Epoch :  9 Step :  1731 loss :  72.9299018333
Epoch :  9 Step :  1732 loss :  50.6366024225
Epoch :  9 Step :  1733 loss :  51.5485861047
Epoch :  9 Step :  1734 loss :  52.8089469223
Epoch :  9 Step :  1735 loss :  56.1271808436
Epoch :  9 Step :  1736 loss :  60.7251571857
Epoch :  9 Step :  1737 loss :  42.1651506805
Epoch :  9 Step :  1738 loss :  57.1264356369
Epoch :  9 Step :  1739 loss :  86.4512521601
Epoch :  9 Step :  1740 loss :  61.4347509329
Epoch :  9 Step :  1741 loss :  48.3823545294
Epoch :  9 Step :  1742 loss :  95.6649733334
Epoch :  9 Step :  1743 loss :  75.1143590309
Epoch :  9 Step :  1744 loss :  45.6803705022
Epoch :  9 Step :  1745 loss :  34.6370447794
Epoch :  9 Step :  1746 loss :  57

Epoch :  9 Step :  1905 loss :  97.13471895
Epoch :  9 Step :  1906 loss :  76.4282836168
Epoch :  9 Step :  1907 loss :  47.9922028916
Epoch :  9 Step :  1908 loss :  45.0567911154
Epoch :  9 Step :  1909 loss :  45.8337283726
Epoch :  9 Step :  1910 loss :  59.1828449781
Epoch :  9 Step :  1911 loss :  42.4225325913
Epoch :  9 Step :  1912 loss :  44.6509130961
Epoch :  9 Step :  1913 loss :  36.2003721183
Epoch :  9 Step :  1914 loss :  47.6611552446
Epoch :  9 Step :  1915 loss :  45.1867534493
Epoch :  9 Step :  1916 loss :  57.9220263663
Epoch :  9 Step :  1917 loss :  41.2056113123
Epoch :  9 Step :  1918 loss :  45.0994180014
Epoch :  9 Step :  1919 loss :  65.4191614029
Epoch :  9 Step :  1920 loss :  44.0015976232
Epoch :  9 Step :  1921 loss :  74.3014983743
Epoch :  9 Step :  1922 loss :  60.5912669822
Epoch :  9 Step :  1923 loss :  41.9518339271
Epoch :  9 Step :  1924 loss :  33.2804968126
Epoch :  9 Step :  1925 loss :  40.0058244991
Epoch :  9 Step :  1926 loss :  54.8

Epoch :  9 Step :  2088 loss :  34.5915154395
Epoch :  9 Step :  2089 loss :  86.3448320323
Epoch :  9 Step :  2090 loss :  46.3468482263
Epoch :  9 Step :  2091 loss :  44.3683733573
Epoch :  9 Step :  2092 loss :  51.1351009964
Epoch :  9 Step :  2093 loss :  45.6699640243
Epoch :  9 Step :  2094 loss :  44.5700750461
Epoch :  9 Step :  2095 loss :  41.9163741521
Epoch :  9 Step :  2096 loss :  49.775468942
Epoch :  9 Step :  2097 loss :  36.3717245788
Epoch :  9 Step :  2098 loss :  43.1354339293
Epoch :  9 Step :  2099 loss :  79.6827541904
Epoch :  9 Step :  2100 loss :  59.9697812276
Epoch :  9 Step :  2101 loss :  57.7704217405
Epoch :  9 Step :  2102 loss :  65.3560829856
Epoch :  9 Step :  2103 loss :  78.029952516
Epoch :  9 Step :  2104 loss :  54.4945891396
Epoch :  9 Step :  2105 loss :  69.311810034
Epoch :  9 Step :  2106 loss :  71.6973215652
Epoch :  9 Step :  2107 loss :  47.6214523542
Epoch :  9 Step :  2108 loss :  50.3701242546
Epoch :  9 Step :  2109 loss :  54.88

In [416]:
100-np.count_nonzero(model.predict(validating_data[1000:1100])-validating_data_class[1000:1100])

/home/mak/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys


53

In [418]:
kp=0
for i,j in zip(model.weights,model.biases):
    np.save('./temp/weights_'+str(kp),i)
    np.save('./temp/biases_'+str(kp),j)
    k
    

In [414]:
validating_data_class[:10]

array([0, 8, 0, 6, 5, 8, 0, 4, 7, 8])